# Assign7: RNN

In [1]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

### Reading in training data

In [2]:
f = open("./Train_Arabic_Digit.txt", "r")

In [3]:
dataset = []
point = []
for line in f:
    if line.isspace() == True and len(point) > 0:
        dataset.append(copy.deepcopy(point))
        point = []
    else:
        if line.isspace() == False:
            line_list = line.strip().split()
            line_float = [ float(x) for x in line_list ]
            point.append(line_float)
dataset.append(copy.deepcopy(point))

In [4]:
f.close()

In [5]:
len(dataset)

6600

### Reading in testing data

In [6]:
f = open("./Test_Arabic_Digit.txt", "r")

In [7]:
testset = []
point = []
for line in f:
    if line.isspace() == True and len(point) > 0:
        testset.append(copy.deepcopy(point))
        point = []
    else:
        if line.isspace() == False:
            line_list = line.strip().split()
            line_float = [ float(x) for x in line_list ]
            point.append(line_float)
testset.append(copy.deepcopy(point))

In [8]:
f.close()

In [9]:
len(testset)

2200

### Making the ground truth labels according to documentation

In [10]:
y_train = []
y_test = []
for i in range(10):
    for j in range(660):
        y_train.append(i)
        
for i in range(10):
    for j in range(220):
        y_test.append(i)

In [11]:
len(y_train)

6600

In [12]:
len(y_test)

2200

### Create One-Hot Encodings

In [13]:
# create a one hot encoding of the target labels
y = np.zeros( (len(y_train), max(y_train) + 1) )
y[np.arange(len(y_train)), y_train] = 1
y_train = copy.deepcopy(y)

In [14]:
# create a one hot encoding of the target labels
y = np.zeros( (len(y_test), max(y_test) + 1) )
y[np.arange(len(y_test)), y_test] = 1
y_test = copy.deepcopy(y)

# RNN Function

### ReLU activation function

In [46]:
def relu(x):
    return (np.maximum(0, x))

In [59]:
def RNN_Training(x, y):
    # define some constants
    maxiter = 40
    d = 13
    m = 16
    p = 10
    eta = 1e-5
    # hidden layer activation function - ReLU
    # output layer activation function - softmax
    
    # initialize bias vectors
    bh = np.random.rand(m)
    bh = bh * 0.01
    bo = np.random.rand(p)
    bo = bo * 0.01
    #initialize weight matrices
    Wi = np.random.rand(d, m)
    Wi = Wi * 0.01
    Wh = np.random.rand(m, m)
    Wh = Wh * 0.01
    Wo = np.random.rand(m, p)
    Wo = Wo * 0.01
    # iteration counter
    r = 0
    while (r < maxiter):
        
        # iterate through dataset in random order
        s = list(range(len(x)))
        random.shuffle(s)
        for i in s:
            sequence = x[i]
            tau = len(sequence)
            
            # initialize hidden state
            h = np.zeros((tau, m))
            # iteratively calculate hidden state
            for j in range(tau):
                t1 = np.dot(Wi.T, sequence[j]) + np.dot(Wh.T, h[j-1]) + bh
                h[j] = relu(t1)
            # compute output layer
            o = softmax(np.dot(Wo.T, h[tau-1]) + bo)
                
        
        
        
        r += 1
    
    
    

In [60]:
RNN_Training(dataset, y_train)

KeyboardInterrupt: 